<H1> Battle of the Neighborhoods <H1>

                     Daniele Rizzi

<h3> Introduction and description of business problem <h3>

Opening a restaurant may be a difficult project, due to many possible errors that the owner may encounter in this phase. Luckily, Data Science can help to simplify this task and help the owner to decide where to open a restaurant and which kind of food may be the best to serve the customers.

In this project I would like to help a future owner to decide which is the best spot to open an Italian Restaurant in Toronto.

This project is useful for restaurant owners / investors that want to get in food businesses; a restaurant in a good position will bring great benefits to both investors and owners since it will attract many people.

<h3> Data <h3>

The work will start by understaning where are located other Italian restaurants in Toronto, basing on the idea that people preferentially go out for lunch and dinner in locations where there are other type of locals.
Then we will investigate if some kind there is a correlation between some activities and restaurants in these neighborhoods, to evaluate if some activities can be found more frequently near restaurants or not; this will help us to identify possible choices for a restaurant where there is less competitivity, due to the fewer number of restaurants, but the possibility to attract customers is high.

<h3> Methodology <h3>

First we collect the data necessary to set up the model. I have already created a database of Toronto locations in another notebook (reference https://github.com/danielerizzi/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb) that will be uploaded into this notebook.
We will use these data along with geospatial and Foursquare data to define the area of interest by looking at where the majority of Italian Restaurants are located in Toronto. After this first stage, we will use the data to understand which are the most frequent venues near Italian Restaurants, and use these data to create a simple graphic from which we will be able to define the best location for the Italian Restaurant.

First, let's install some useful libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::numba==0.42.0=py36h962f231_0
  - defaults/linux-64::blaze==0.11.3=py36_0
done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tbb-2019.9                 |       hc9558a2_1         1.4 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.4 MB

The following NEW packages will be INSTALLED:

  tbb                conda-forge/linux-64::tbb-2019.9-hc9558a2_1



tbb-2019.9           | 1.4 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solv

In [3]:
!conda update -n base -c defaults conda --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    chardet-3.0.4              |        py37_1003         174 KB
    libstdcxx-ng-9.1.0         |       hdf63c60_0         3.1 MB
    pycosat-0.6.3              |   py37h7b6447c_0          82 KB
    pycparser-2.19             |           py37_0         171 KB
    pysocks-1.7.1              |           py37_0          31 KB
    requests-2.22.0            |           py37_1          92 KB
    ruamel_yaml-0.15.87        |   py37h7b6447c_0         256 KB
    urllib3-1.25.7             |           py37_0         169 KB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following packages will be REMOVED:

  _libgcc_mutex-0.1-main
  conda-env-2.6.0

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


Now we import csv data of Toronto from file (reference http://cocl.us/Geospatial_data)

In [5]:
!wget -q -O 'Toronto_data.csv'  http://cocl.us/Geospatial_data
geo = pd.read_csv('Toronto_data.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
geo.shape

(103, 3)

Now we import csv data of Toronto already created (reference https://github.com/danielerizzi/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb)

In [18]:
# The code was removed by Watson Studio for sharing.

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned\n


Let's rename "Postal Code" column to merge the two dataframes found in a single one.

In [19]:
geo = geo.rename(columns={'Postal Code': 'Postcode'})

toronto = pd.merge(df_merged, geo, on='Postcode', how='outer')
toronto.head(15)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned\n,43.662301,-79.389494
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North\n,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson\n, Garden District\n",43.657162,-79.378937


Let's download json data of Toronto that will be used to generate graphics

In [20]:
!wget -q -O 'toronto_data.json' http://adamw523.com/toronto-geojson/
print('Data downloaded!')

Data downloaded!


In [21]:
CLIENT_ID = 'HP01PRUXPJX5YJRMN3U1XHS22FAN10F00CWHKD3XUN2FIAWG' # your Foursquare ID
CLIENT_SECRET = '0WQU3GJFSG3I0Z2TD1GEJ0EKIH5N2ZAJUQLNG3EVBR2V0IYK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 10000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HP01PRUXPJX5YJRMN3U1XHS22FAN10F00CWHKD3XUN2FIAWG
CLIENT_SECRET:0WQU3GJFSG3I0Z2TD1GEJ0EKIH5N2ZAJUQLNG3EVBR2V0IYK


In [22]:
address_TO = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_agent")
location = geolocator.geocode(address_TO)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


We now create a map of Toronto on which we will work

In [23]:
# create map of New York using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

We use a function to get names and information on venues in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=toronto['Postcode'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )
toronto_venues.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Now we create a database of venues category Italian Restaurant

In [27]:
toronto_venues_it_rest = toronto_venues[toronto_venues['Venue Category'] == 'Italian Restaurant']
toronto_venues_it_rest.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
40,M5A,43.654260,-79.360636,Archeo,43.650667,-79.359431,Italian Restaurant
70,M7A,43.662301,-79.389494,Mercatto,43.660391,-79.387664,Italian Restaurant
184,M5B,43.657162,-79.378937,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,Italian Restaurant
186,M5B,43.657162,-79.378937,Trattoria Mercatto,43.654453,-79.380974,Italian Restaurant
234,M3C,43.725900,-79.340923,Sorento Restaurant,43.726575,-79.341989,Italian Restaurant


Now we need to understand which is the most common Venue around an Italian Restaurant.
This will help us in defining in a more precise way the area in which an Italian Restaurant is more likely to have success and customers.

To do so, we first create a dataframe in which we calculate the distance between Italian Restaurants and other Venues.

In [28]:
from math import radians, cos, sin, asin, sqrt

#define a function to evaluate distance between venues
#refer https://en.wikipedia.org/wiki/Haversine_formula

def haversine(lon1, lat1, lon2, lat2):
  # convert decimal degrees to radians 
  lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
  # haversine formula 
  distance_lon = lon2 - lon1 
  distance_lat = lat2 - lat1
  a = sin(distance_lat/2)**2 + cos(lat1) * cos(lat2) * sin(distance_lon/2)**2
  earth_r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    
  return 2 * asin(sqrt(a)) * earth_r #result in km

restaurant_venues = pd.DataFrame(columns = ['Restaurant','Venue type', 'Restaurant longitude', 'Restaurant latitude', 'Venue longitude','Venue latitude', 'Distance'])

for rest, lng, lat in zip(toronto_venues_it_rest['Venue Category'], toronto_venues_it_rest['Venue Longitude'], toronto_venues_it_rest['Venue Latitude']):
    for venue, lng2, lat2 in zip(toronto_venues['Venue Category'], toronto_venues['Venue Longitude'], toronto_venues['Venue Latitude']):
        distance = haversine(lng, lat, lng2, lat2)
        restaurant_venues = restaurant_venues.append({'Restaurant': rest, 
                                                      'Venue type': venue, 
                                                      'Restaurant longitude': lng,
                                                      'Restaurant latitude': lat,
                                                      'Venue longitude': lng2,
                                                      'Venue latitude': lat2,
                                                      'Distance': distance
                                                     }, ignore_index=True)

print('----------------Database created!----------------')
        

----------------Database created!----------------


In [30]:
restaurant_venues.shape

(101660, 7)

We then filter the database by removing informations not useful with the following criteria

1. Rows where distance is equal to 0 (we have evaluated distance between a venue and itself)
2. Rows where distance is greater than 200 meters

After that we sort the values in descending order, in order to find the top venues located near Italian Restaurants

In [31]:
restaurant_venues_mod = restaurant_venues[restaurant_venues['Distance'] != 0] #remove 0 distance rows
restaurant_venues_mod200 = restaurant_venues_mod[restaurant_venues['Distance'] < 0.2] #keep only the venues up to 200 meters
restaurant_venues_mod200.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


,Restaurant,Venue type,Restaurant longitude,Restaurant latitude,Venue longitude,Venue latitude,Distance
16,Italian Restaurant,Historic Site,-79.359431,43.650667,-79.359323,43.650244,0.047816
18,Italian Restaurant,Chocolate Shop,-79.359431,43.650667,-79.358127,43.650622,0.105019
20,Italian Restaurant,Dessert Shop,-79.359431,43.650667,-79.360723,43.650067,0.123549
21,Italian Restaurant,Bakery,-79.359431,43.650667,-79.359539,43.650574,0.013564
22,Italian Restaurant,Coffee Shop,-79.359431,43.650667,-79.361442,43.649963,0.179771


In [32]:
restaurant_venues_mod200.shape

(1385, 7)

In [33]:
restaurant_venues_mod200_sorted = restaurant_venues_mod200.groupby('Venue type').count().sort_values(['Distance'], ascending=[False])
restaurant_venues_mod200_sorted

,Restaurant,Restaurant longitude,Restaurant latitude,Venue longitude,Venue latitude,Distance
Venue type,,,,,,
Coffee Shop,141,141,141,141,141,141
American Restaurant,66,66,66,66,66,66
Bakery,61,61,61,61,61,61
Café,56,56,56,56,56,56
Restaurant,44,44,44,44,44,44
Burger Joint,43,43,43,43,43,43
Gastropub,41,41,41,41,41,41
Beer Bar,40,40,40,40,40,40
Seafood Restaurant,40,40,40,40,40,40


We want to select only the venues that are not restaurants. 
The first three venues are

1. Coffee Shop
2. Bar
3. Hotel

We will use these to better define where to open the restaurant

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues_it_rest['Postcode']
toronto_onehot['Neighborhood Latitude'] = toronto_venues_it_rest['Neighborhood Latitude'] 
toronto_onehot['Neighborhood Longitude'] = toronto_venues_it_rest['Neighborhood Longitude'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot_italian = toronto_onehot[toronto_onehot['Italian Restaurant'] != 0]
toronto_onehot_italian.reset_index()
toronto_onehot_italian.head()

,Neighborhood Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Postcode,Neighborhood Latitude
40,-79.360636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

We use the data otained to create a Cluster Map of Toronto showing the number of Italian Restaurants per each location

In [35]:
from folium import plugins
italian_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# instantiate a mark cluster object for the incidents in the dataframe
italian_restaurant = plugins.MarkerCluster().add_to(italian_restaurant_map)

# add markers to map
for lat, lng, label in zip(toronto_onehot_italian['Neighborhood Latitude'], toronto_onehot_italian['Neighborhood Longitude'], toronto_onehot_italian['Postcode']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(italian_restaurant)
    
italian_restaurant_map

As we can see from these data, 22 Italian Restaurants may be found between Toronto Union Station and the University of Toronto. This tells us that this is one of the most frequented zone in Toronto, therefore this is one of the best spots where to open an Italian Restaurant.

We can then use the data about the other venues near Italian Restaurants to better define the area on which focus our attention.

We now introduce the search query to search in Foursquare data

In [36]:
search_query = 'Italian Restaurant'
radius = 50000
#print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HP01PRUXPJX5YJRMN3U1XHS22FAN10F00CWHKD3XUN2FIAWG&client_secret=0WQU3GJFSG3I0Z2TD1GEJ0EKIH5N2ZAJUQLNG3EVBR2V0IYK&ll=43.653963,-79.387207&v=20180604&query=Italian Restaurant&radius=50000&limit=10000'

And check the obtained dataframe

In [37]:
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)
#dataframe.drop(dataframe[(dataframe['location.postalCode'] == 'NaN')].index(), inplace=True)

dataframe.shape

(50, 19)

In [38]:
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4e74ce151838f918898efe72,6350 Tomken Rd.,CA,Mississauga,Canada,Tristar,22619,"[6350 Tomken Rd. (Tristar), Mississauga ON, Ca...","[{'label': 'display', 'lat': 43.65285913569359...",43.652859,-79.668040,NaN,NaN,ON,Roma Italian Restaurant,v-1577102315,NaN
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4de024f0b0fbe2cfa5fee3c4,NaN,CA,Toronto,Canada,NaN,3572,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67656199554484...",43.676562,-79.355699,NaN,NaN,ON,Florentina's Italian Restaurant,v-1577102315,NaN
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4b4f6a73f964a520960527e3,24-40 Bradwick Dr,CA,Vaughan,Canada,btw Keele St. & Dufferin St.,19907,[24-40 Bradwick Dr (btw Keele St. & Dufferin S...,"[{'label': 'display', 'lat': 43.8182382, 'lng'...",43.818238,-79.485024,NaN,L4K 1K9,ON,Junnio's Italian Restaurant,v-1577102315,NaN
3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4b199b46f964a5205be023e3,2625 Weston,CA,Toronto,Canada,401,13289,"[2625 Weston (401), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.71194605276710...",43.711946,-79.531510,NaN,NaN,ON,Jolly II Italian Restaurant,v-1577102315,NaN
4,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4b107754f964a520147123e3,4505 Sheppard Ave E,CA,Scarborough,Canada,NaN,17869,"[4505 Sheppard Ave E, Scarborough ON M1S 1V3, ...","[{'label': 'display', 'lat': 43.788071, 'lng':...",43.788071,-79.265134,NaN,M1S 1V3,ON,Joey Bravo's Italian Restaurant,v-1577102315,NaN
5,"[{'id': '56aa371be4b08b9a8d573550', 'name': 'F...",False,55e549c8498ea9b7acd67e8b,6720 Davand Drive,CA,Mississauga,Canada,NaN,22657,"[6720 Davand Drive, Mississauga ON L5T 2K7, Ca...","[{'label': 'display', 'lat': 43.66758, 'lng': ...",43.667580,-79.667920,NaN,L5T 2K7,ON,Il Porcellino Italian Restaurant And Catering,v-1577102315,NaN
6,[],False,4d5577519e508cfa9c72009b,Derry Road West,CA,Mississauga,Canada,NaN,21586,"[Derry Road West, Mississauga ON, Canada]","[{'label': 'display', 'lat': 43.703068, 'lng':...",43.703068,-79.646597,NaN,NaN,ON,Buda's Italian Restaurant,v-1577102315,NaN
7,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4c7d330c5af8b60cb8d38e10,NaN,CA,NaN,Canada,NaN,23253,[Canada],"[{'label': 'display', 'lat': 43.68860510830914...",43.688605,-79.672008,NaN,NaN,NaN,Mia Italian Restaurant,v-1577102315,NaN
8,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4e24ce2bd164e1d5a512bfe6,NaN,CA,NaN,Canada,NaN,27531,"[Ontario, Canada]","[{'label': 'display', 'lat': 43.88753489635225...",43.887535,-79.499824,NaN,NaN,Ontario,Marchellos italian restaurant,v-1577102315,NaN
9,[],False,4b53882ef964a5200fa127e3,NaN,CA,NaN,Canada,NaN,32265,[Canada],"[{'label': 'display', 'lat': 43.44640202412109...",43.446402,-79.666352,NaN,NaN,NaN,Roccos italian restaurant,v-1577102315,NaN


As we can see, the name of the columns are not user friendly, so we work on this dataframe to change the column names.

In [39]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Roma Italian Restaurant,Indian Restaurant,6350 Tomken Rd.,CA,Mississauga,Canada,Tristar,22619,"[6350 Tomken Rd. (Tristar), Mississauga ON, Ca...","[{'label': 'display', 'lat': 43.65285913569359...",43.652859,-79.668040,NaN,NaN,ON,4e74ce151838f918898efe72
1,Florentina's Italian Restaurant,Italian Restaurant,NaN,CA,Toronto,Canada,NaN,3572,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67656199554484...",43.676562,-79.355699,NaN,NaN,ON,4de024f0b0fbe2cfa5fee3c4
2,Junnio's Italian Restaurant,Restaurant,24-40 Bradwick Dr,CA,Vaughan,Canada,btw Keele St. & Dufferin St.,19907,[24-40 Bradwick Dr (btw Keele St. & Dufferin S...,"[{'label': 'display', 'lat': 43.8182382, 'lng'...",43.818238,-79.485024,NaN,L4K 1K9,ON,4b4f6a73f964a520960527e3
3,Jolly II Italian Restaurant,Italian Restaurant,2625 Weston,CA,Toronto,Canada,401,13289,"[2625 Weston (401), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.71194605276710...",43.711946,-79.531510,NaN,NaN,ON,4b199b46f964a5205be023e3
4,Joey Bravo's Italian Restaurant,American Restaurant,4505 Sheppard Ave E,CA,Scarborough,Canada,NaN,17869,"[4505 Sheppard Ave E, Scarborough ON M1S 1V3, ...","[{'label': 'display', 'lat': 43.788071, 'lng':...",43.788071,-79.265134,NaN,M1S 1V3,ON,4b107754f964a520147123e3


We create a new column named "Postcode" that will be used for the graphics.

In [40]:
#Keep only first 3 of Postcode
dataframe_filtered['Postcode'] = dataframe_filtered['postalCode'].str[:3]
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
0,Roma Italian Restaurant,Indian Restaurant,6350 Tomken Rd.,CA,Mississauga,Canada,Tristar,22619,"[6350 Tomken Rd. (Tristar), Mississauga ON, Ca...","[{'label': 'display', 'lat': 43.65285913569359...",43.652859,-79.668040,NaN,NaN,ON,4e74ce151838f918898efe72,NaN
1,Florentina's Italian Restaurant,Italian Restaurant,NaN,CA,Toronto,Canada,NaN,3572,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67656199554484...",43.676562,-79.355699,NaN,NaN,ON,4de024f0b0fbe2cfa5fee3c4,NaN
2,Junnio's Italian Restaurant,Restaurant,24-40 Bradwick Dr,CA,Vaughan,Canada,btw Keele St. & Dufferin St.,19907,[24-40 Bradwick Dr (btw Keele St. & Dufferin S...,"[{'label': 'display', 'lat': 43.8182382, 'lng'...",43.818238,-79.485024,NaN,L4K 1K9,ON,4b4f6a73f964a520960527e3,L4K
3,Jolly II Italian Restaurant,Italian Restaurant,2625 Weston,CA,Toronto,Canada,401,13289,"[2625 Weston (401), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.71194605276710...",43.711946,-79.531510,NaN,NaN,ON,4b199b46f964a5205be023e3,NaN
4,Joey Bravo's Italian Restaurant,American Restaurant,4505 Sheppard Ave E,CA,Scarborough,Canada,NaN,17869,"[4505 Sheppard Ave E, Scarborough ON M1S 1V3, ...","[{'label': 'display', 'lat': 43.788071, 'lng':...",43.788071,-79.265134,NaN,M1S 1V3,ON,4b107754f964a520147123e3,M1S


We now check the data obtained

In [41]:
dataframe_filtered.groupby('categories').count()

,name,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
categories,,,,,,,,,,,,,,,,
American Restaurant,3,3,3,3,3,0,3,3,3,3,3,0,2,3,3,2
Bar,2,2,2,2,2,1,2,2,2,2,2,0,1,2,2,1
Breakfast Spot,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Caribbean Restaurant,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Chinese Restaurant,6,6,6,6,6,3,6,6,6,6,6,0,4,6,6,4
Dim Sum Restaurant,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2
Embassy / Consulate,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0
English Restaurant,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Food Service,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1


We see from table above we see that from the search query we have also obtained other categories we are not interested in. Therefore we filter again the dataframe to obtain the final version containing only Italian Restaurants.

In [42]:
italian_restaurant_df = dataframe_filtered.loc[(dataframe_filtered['categories'] == 'Italian Restaurant')].reset_index()
italian_restaurant_df

,index,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
0,1,Florentina's Italian Restaurant,Italian Restaurant,NaN,CA,Toronto,Canada,NaN,3572,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67656199554484...",43.676562,-79.355699,NaN,NaN,ON,4de024f0b0fbe2cfa5fee3c4,NaN
1,3,Jolly II Italian Restaurant,Italian Restaurant,2625 Weston,CA,Toronto,Canada,401,13289,"[2625 Weston (401), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.71194605276710...",43.711946,-79.531510,NaN,NaN,ON,4b199b46f964a5205be023e3,NaN
2,7,Mia Italian Restaurant,Italian Restaurant,NaN,CA,NaN,Canada,NaN,23253,[Canada],"[{'label': 'display', 'lat': 43.68860510830914...",43.688605,-79.672008,NaN,NaN,NaN,4c7d330c5af8b60cb8d38e10,NaN
3,8,Marchellos italian restaurant,Italian Restaurant,NaN,CA,NaN,Canada,NaN,27531,"[Ontario, Canada]","[{'label': 'display', 'lat': 43.88753489635225...",43.887535,-79.499824,NaN,NaN,Ontario,4e24ce2bd164e1d5a512bfe6,NaN
4,10,Nino's Authentic Italian Restaurant,Italian Restaurant,438 Kerr St,CA,Oakville,Canada,NaN,33376,"[438 Kerr St, Oakville ON L6K 3C4, Canada]","[{'label': 'display', 'lat': 43.44530127192647...",43.445301,-79.684267,NaN,L6K 3C4,ON,55bcde11498e244a3a859792,L6K
5,12,Focacia's Italian Restaurant,Italian Restaurant,NaN,CA,NaN,Canada,NaN,37140,[Canada],"[{'label': 'display', 'lat': 43.853653, 'lng':...",43.853653,-79.017173,NaN,NaN,NaN,4c9a4464a004a1cd97de466e,NaN
6,13,Scaddabush Italian Kitchen & Bar,Italian Restaurant,"382 Yonge Street, Unit #7",CA,Toronto,Canada,Gerrard,652,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...","[{'label': 'display', 'lat': 43.65892029202872...",43.658920,-79.382891,NaN,M5B 1S8,ON,52f6816f11d24a43115dc834,M5B
7,14,Fabbrica Rustic Italian,Italian Restaurant,66 Wellington St W,CA,Toronto,Canada,NaN,877,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]","[{'label': 'display', 'lat': 43.647161, 'lng':...",43.647161,-79.381691,NaN,M5K 1E7,ON,5b897e92db1d81002c91df8c,M5K
8,15,The Fresh Italian,Italian Restaurant,NaN,CA,Toronto,Canada,NaN,127,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65499143746528...",43.654991,-79.387897,NaN,NaN,ON,51bf3866498e55ee55df8db0,NaN
9,21,The Express Italian Restaurant,Italian Restaurant,1477 Lakeshore Rd,CA,Burlington,Canada,NaN,49393,"[1477 Lakeshore Rd, Burlington ON L7S 1B5, Can...","[{'label': 'display', 'lat': 43.32437, 'lng': ...",43.324370,-79.796670,NaN,L7S 1B5,ON,5b394376c66666002c5e54ef,L7S


Now let's use the data we have to create a Marker Cluster map to see where to focus our attention in the data analysis.

In [43]:
from folium import plugins
italian_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# instantiate a mark cluster object for the incidents in the dataframe
#italian_restaurant_df_marker = plugins.MarkerCluster().add_to(italian_restaurant_map)

# add markers to map
for lat, lng, label in zip(italian_restaurant_df['lat'], italian_restaurant_df['lng'], italian_restaurant_df['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(italian_restaurant_map)
    
italian_restaurant_map

From this graphic we see that the majority of the Italian Restaurants are located between Toronto Union Station and college street.

Now we check if the top venues that are commonly found around Italian Restaurants can help us in defining interesting areas

In [44]:
#First search: Coffee Shop
search_query = 'Coffee Shop'
radius = 50000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

coffee_shop = requests.get(url).json()
# assign relevant part of JSON to venues
venues_coffee_shop = coffee_shop['response']['venues']
# tranform venues into a dataframe
coffee_shop = json_normalize(venues_coffee_shop)
#dataframe.drop(dataframe[(dataframe['location.postalCode'] == 'NaN')].index(), inplace=True)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in coffee_shop.columns if col.startswith('location.')] + ['id']
coffee_shop = coffee_shop.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
coffee_shop['categories'] = coffee_shop.apply(get_category_type, axis=1)

# clean column names by keeping only last term
coffee_shop.columns = [column.split('.')[-1] for column in coffee_shop.columns]

#Keep only first 3 of Postcode
coffee_shop['Postcode'] = coffee_shop['postalCode'].str[:3]

coffee_shop.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
0,Bluestone Lane Queen Station Coffee Shop,Café,2 Queen Street East,CA,Toronto,Canada,NaN,686,"[2 Queen Street East, Toronto ON M5C 3G5, Canada]","[{'label': 'display', 'lat': 43.6527958, 'lng'...",43.652796,-79.378843,NaN,M5C 3G5,ON,5d493dc5735c2d0007bc3966,M5C
1,Super Jet International Coffee Shop,Coffee Shop,267 College St.,CA,Toronto,Canada,Spadina Ave,1107,"[267 College St. (Spadina Ave), Toronto ON M5T...","[{'label': 'display', 'lat': 43.657971, 'lng':...",43.657971,-79.399795,NaN,M5T 1R5,ON,5ae32b6412c8f0002c2b03e7,M5T
2,GRIP 6th Floor Coffee Shop,Coffee Shop,179 John St.,CA,Toronto,Canada,at Queen St.,461,"[179 John St. (at Queen St.), Toronto ON M5T 1...","[{'label': 'display', 'lat': 43.65103493678703...",43.651035,-79.391256,NaN,M5T 1X4,ON,4c6bd9de69b4ef3b4d51474e,M5T
3,Niagara coffee shop,Coffee Shop,Niagara St.,CA,Toronto,Canada,NaN,1821,"[Niagara St., Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64188299139057...",43.641883,-79.402463,NaN,NaN,ON,4bc796086501c9b6a1ba3e29,NaN
4,Luis Coffee Shop,Coffee Shop,235 Augusta St,CA,Toronto,Canada,Baldwin & Augusta,1193,"[235 Augusta St (Baldwin & Augusta), Toronto O...","[{'label': 'display', 'lat': 43.654575, 'lng':...",43.654575,-79.402006,NaN,M5T 1M1,ON,4ae92277f964a52072b421e3,M5T


In [45]:
coffee_shop.groupby('categories').count()

,name,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
categories,,,,,,,,,,,,,,,,
Café,8,3,8,6,8,3,8,8,8,8,8,0,2,6,8,2
Coffee Shop,34,29,34,30,34,23,34,34,34,34,34,1,20,30,34,20
College Cafeteria,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1
Gift Shop,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1
Lounge,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1
Pop-Up Shop,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Pub,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0
Restaurant,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1


In [46]:
#keep only category Coffee Shop
coffee_shop = coffee_shop.loc[(coffee_shop['categories'] == 'Coffee Shop')].reset_index()
coffee_shop.head()

,index,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
0,1,Super Jet International Coffee Shop,Coffee Shop,267 College St.,CA,Toronto,Canada,Spadina Ave,1107,"[267 College St. (Spadina Ave), Toronto ON M5T...","[{'label': 'display', 'lat': 43.657971, 'lng':...",43.657971,-79.399795,NaN,M5T 1R5,ON,5ae32b6412c8f0002c2b03e7,M5T
1,2,GRIP 6th Floor Coffee Shop,Coffee Shop,179 John St.,CA,Toronto,Canada,at Queen St.,461,"[179 John St. (at Queen St.), Toronto ON M5T 1...","[{'label': 'display', 'lat': 43.65103493678703...",43.651035,-79.391256,NaN,M5T 1X4,ON,4c6bd9de69b4ef3b4d51474e,M5T
2,3,Niagara coffee shop,Coffee Shop,Niagara St.,CA,Toronto,Canada,NaN,1821,"[Niagara St., Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64188299139057...",43.641883,-79.402463,NaN,NaN,ON,4bc796086501c9b6a1ba3e29,NaN
3,4,Luis Coffee Shop,Coffee Shop,235 Augusta St,CA,Toronto,Canada,Baldwin & Augusta,1193,"[235 Augusta St (Baldwin & Augusta), Toronto O...","[{'label': 'display', 'lat': 43.654575, 'lng':...",43.654575,-79.402006,NaN,M5T 1M1,ON,4ae92277f964a52072b421e3,M5T
4,5,Buro 53 Coffee Shop,Coffee Shop,53 Bloor St E,CA,Toronto,Canada,Yonge,1850,"[53 Bloor St E (Yonge), Toronto ON M4W 1A9, Ca...","[{'label': 'display', 'lat': 43.67053207847662...",43.670532,-79.385293,NaN,M4W 1A9,ON,590c8627a423623a3e5f534e,M4W


In [47]:
italian_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# instantiate a mark cluster object for the incidents in the dataframe
#italian_restaurant_df_marker = plugins.MarkerCluster().add_to(italian_restaurant_map)

# add markers to map
for lat, lng, label in zip(italian_restaurant_df['lat'], italian_restaurant_df['lng'], italian_restaurant_df['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(italian_restaurant_map)

# add markers of Coffee Shops to map
for lat, lng, label in zip(coffee_shop['lat'], coffee_shop['lng'], coffee_shop['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color = 'green'),
        popup=label,
    ).add_to(italian_restaurant_map)
    
italian_restaurant_map

From above graphic we see that good zone in which there are Coffee Shops and not too many Italian Restaurants is the rectangle between "Osgoode", "St. Patrick", "Dundas" and "Queen".

Let's try to refine our search by looking at other venues

In [48]:
#Second search: Bar
search_query = 'Bar'
radius = 50000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

bar = requests.get(url).json()
# assign relevant part of JSON to venues
venues_bar = bar['response']['venues']
# tranform venues into a dataframe
bar = json_normalize(venues_bar)
#dataframe.drop(dataframe[(dataframe['location.postalCode'] == 'NaN')].index(), inplace=True)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in bar.columns if col.startswith('location.')] + ['id']
bar = bar.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
bar['categories'] = bar.apply(get_category_type, axis=1)

# clean column names by keeping only last term
bar.columns = [column.split('.')[-1] for column in bar.columns]

#Keep only first 3 of Postcode
bar['Postcode'] = bar['postalCode'].str[:3]

bar.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
0,Bar+ Karaoke Lounge,Karaoke Bar,360 Yonge St.,CA,Toronto,Canada,at Elm St.,647,"[360 Yonge St. (at Elm St.), Toronto ON M5B 1S...","[{'label': 'display', 'lat': 43.65833815125888...",43.658338,-79.381902,NaN,M5B 1S5,ON,4ad949def964a520991921e3,M5B
1,Quinn's Steakhouse & Bar,Steakhouse,96 Richmond Street West,CA,Toronto,Canada,at Bay,459,"[96 Richmond Street West (at Bay), Toronto ON ...","[{'label': 'display', 'lat': 43.65119745750837...",43.651197,-79.382976,Financial District,M5H 2A3,ON,4b3db5e4f964a520709625e3,M5H
2,Rialto Espresso Bar,Café,266 Adelaide Street West,CA,Toronto,Canada,John Street,689,"[266 Adelaide Street West (John Street), Toron...","[{'label': 'display', 'lat': 43.648059, 'lng':...",43.648059,-79.389791,NaN,NaN,ON,54f5f80e498e300bbba1e5be,NaN
3,St. Louis Bar and Grill,Bar,595 Bay St.,CA,Toronto,Canada,Atrium on Bay,448,"[595 Bay St. (Atrium on Bay), Toronto ON M5G 2...","[{'label': 'display', 'lat': 43.65632495785165...",43.656325,-79.382693,NaN,M5G 2C2,ON,4af4e6e2f964a52052f721e3,M5G
4,Duke's Refresher + Bar,Bar,"382 Yonge St, Unit 8",CA,Toronto,Canada,at Gerrard St W,655,"[382 Yonge St, Unit 8 (at Gerrard St W), Toron...","[{'label': 'display', 'lat': 43.65897985683036...",43.658980,-79.382949,NaN,M5B 1S8,ON,529fa9e111d2159d1f3caa0e,M5B


In [49]:
bar.groupby('categories').count()

,name,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
categories,,,,,,,,,,,,,,,,
American Restaurant,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3
BBQ Joint,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Bar,5,5,5,5,5,3,5,5,5,5,5,0,4,5,5,4
Beer Bar,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Café,6,6,6,6,6,6,6,6,6,6,6,0,4,6,6,4
Cocktail Bar,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Coffee Shop,4,4,4,4,4,2,4,4,4,4,4,0,3,4,4,3
Hotel Bar,5,5,5,5,5,3,5,5,5,5,5,0,4,5,5,4
Italian Restaurant,3,3,3,3,3,2,3,3,3,3,3,0,3,3,3,3


In [50]:
italian_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# instantiate a mark cluster object for the incidents in the dataframe
#italian_restaurant_df_marker = plugins.MarkerCluster().add_to(italian_restaurant_map)

# add markers to map
for lat, lng, label in zip(italian_restaurant_df['lat'], italian_restaurant_df['lng'], italian_restaurant_df['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(italian_restaurant_map)
    
# add markers of Bars to map
for lat, lng, label in zip(bar['lat'], bar['lng'], bar['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color = 'red'),
        popup=label,
    ).add_to(italian_restaurant_map)
    
italian_restaurant_map

This result seems to give us other inputs; the location of venues that are typically found near Italian Restaurant opens the area expanding it up to Toronto Union Station.

Let's try the third popular venue

In [51]:
#Third search: hotel
search_query = 'Hotel'
radius = 50000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

hotel = requests.get(url).json()
# assign relevant part of JSON to venues
venues_hotel = hotel['response']['venues']
# tranform venues into a dataframe
hotel = json_normalize(venues_hotel)
#dataframe.drop(dataframe[(dataframe['location.postalCode'] == 'NaN')].index(), inplace=True)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in hotel.columns if col.startswith('location.')] + ['id']
hotel = hotel.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
hotel['categories'] = hotel.apply(get_category_type, axis=1)

# clean column names by keeping only last term
hotel.columns = [column.split('.')[-1] for column in hotel.columns]

#Keep only first 3 of Postcode
hotel['Postcode'] = hotel['postalCode'].str[:3]

hotel.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,at York St.,390,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6511441, 'lng'...",43.651144,-79.384329,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3,M5H
1,Chelsea Hotel,Hotel,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,603,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,51d212c3498ebf27dc469bc9,M5G
2,The Grand Hotel & Suites Toronto,Hotel,225 Jarvis St.,CA,Toronto,Canada,at Dundas St. E.,1090,"[225 Jarvis St. (at Dundas St. E.), Toronto ON...","[{'label': 'display', 'lat': 43.65644901207743...",43.656449,-79.374110,NaN,M5B 2C1,ON,4b7d9098f964a52014c72fe3,M5B
3,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,400,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,4b68aed1f964a520de862be3,M5V
4,The Omni King Edward Hotel,Hotel,37 King Street East,CA,Toronto,Canada,bwtn Victoria St. and Toronto St.,1046,[37 King Street East (bwtn Victoria St. and To...,"[{'label': 'display', 'lat': 43.64919051601911...",43.649191,-79.376006,NaN,M5C 1E9,ON,4adf7d0bf964a520127b21e3,M5C


In [52]:
hotel.groupby('categories').count()

,name,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Postcode
categories,,,,,,,,,,,,,,,,
Ballroom,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0
College Residence Hall,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1
Gift Shop,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0
Gym / Fitness Center,1,0,1,0,1,0,1,1,1,1,1,1,0,0,1,0
Hotel,39,36,39,38,39,25,39,39,39,39,39,1,31,38,39,31
Hotel Bar,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Hotel Pool,1,1,1,1,1,0,1,1,1,1,1,0,0,1,1,0
Jazz Club,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
Lounge,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,1


In [53]:
italian_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# instantiate a mark cluster object for the incidents in the dataframe
#italian_restaurant_df_marker = plugins.MarkerCluster().add_to(italian_restaurant_map)

# add markers to map
for lat, lng, label in zip(italian_restaurant_df['lat'], italian_restaurant_df['lng'], italian_restaurant_df['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(italian_restaurant_map)
    
# add markers of Hotels to map
for lat, lng, label in zip(hotel['lat'], hotel['lng'], hotel['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color = 'orange'),
        popup=label,
    ).add_to(italian_restaurant_map)
    
italian_restaurant_map

This last search confirms what we found above; the area in which the most popular venues commonly found near Italian Restaurants is from Queen's Park up to Toronto Union Station.

Let's put everything together.

In [54]:
italian_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# instantiate a mark cluster object for the incidents in the dataframe
#italian_restaurant_df_marker = plugins.MarkerCluster().add_to(italian_restaurant_map)

# add markers to map
for lat, lng, label in zip(italian_restaurant_df['lat'], italian_restaurant_df['lng'], italian_restaurant_df['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(italian_restaurant_map)
    
# add markers of Hotels to map
for lat, lng, label in zip(hotel['lat'], hotel['lng'], hotel['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color = 'orange'),
        popup=label,
    ).add_to(italian_restaurant_map)
    
# add markers of Bars to map
for lat, lng, label in zip(bar['lat'], bar['lng'], bar['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color = 'red'),
        popup=label,
    ).add_to(italian_restaurant_map)
    

# add markers of Coffee Shops to map
for lat, lng, label in zip(coffee_shop['lat'], coffee_shop['lng'], coffee_shop['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color = 'green'),
        popup=label,
    ).add_to(italian_restaurant_map)
    
    
italian_restaurant_map

This picture shows Italian Restaurants and other venues we analysed in a single map zoomed to show the area of interest. As we can see, we can now refine the area suggested for an Italian Restaurant to Adelaide Street West or Adelaide Street East.

<H3> Conclusion and discussion <H3>

From above results we can now conclude that

1. Many popular venues are located in a rectangle defined by Richmond Street West, Church Street, Front Street and Queen Street.
2. In this area there are 3 other Italian Restaurants near University Avenue
3. University Avenue splits the area of interest in 2 sub-areas (East and West)
4. Our suggestion for the location of an Italian Restaurant is then near Adelaide Street West or Adelaide Street East in order to have the Restaurant located near interesting venues, but not too close to other Italian Restaurants that are already well established in the city.

This analysis gave us only a suggestion of a possible location of an Italian Restaurant. The variables we considered were only a few of all the possible ones we may have analyses, such as position of parkings, presence of restaurants serving other cuisine type and cost of the rent. Further analysis including these variables may help us in refining our search more and finding a more suitable location to suggest to our customer.